# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os


# Import API key
from config import api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:

weather_data = pd.read_csv("../WeatherPy/cities.csv") 
weather_data

,Cities,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,hobart,au,-42.88,147.33,66.00,80.0,100.0,1.99
1,port augusta,au,-32.50,137.77,70.00,52.0,0.0,1.99
2,bluff,nz,-46.60,168.33,57.00,86.0,61.0,2.80
3,kapaa,us,22.08,-159.32,68.00,59.0,45.0,7.34
4,kedrovyy,ru,56.17,91.82,23.13,79.0,100.0,12.80
...,...,...,...,...,...,...,...,...
622,musoma,tz,-1.50,33.80,80.60,65.0,20.0,13.87
623,flagstaff,us,35.20,-111.65,39.99,92.0,1.0,1.63
624,saleaula,ws,NaN,NaN,NaN,NaN,NaN,NaN
625,sumbawa,id,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=api_key)

# Store latitude and longitude in locations
locations = weather_data[["Latitude", "Longitude"]]
locations = locations.dropna()   

# Store Humidity in humidity
humidity = weather_data["Humidity (%)"]
humidity = humidity.dropna()


In [16]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

# A max temperature lower than 80 degrees but higher than 70.

# Wind speed less than 10 mph.

# Zero cloudiness.

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.



In [18]:
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed (mph)"] <= 10) & (weather_data["Cloudiness (%)"] == 0) & \
                                   (weather_data["Temperature (F)"] > 70) & (weather_data["Temperature (F)"] < 80)].dropna()

narrowed_city_df

#My base file does not have high temps? What did I do wrong?

,Cities,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
135,bac lieu,vn,9.29,105.72,73.31,93.0,0.0,6.40
165,cidreira,br,-30.18,-50.21,76.35,56.0,0.0,4.34
185,taltal,cl,-25.40,-70.48,72.82,70.0,0.0,7.47
254,mazabuka,zm,-15.86,27.75,76.48,70.0,0.0,7.58
287,ballina,au,-28.87,153.57,73.99,91.0,0.0,10.00
329,henties bay,na,-22.12,14.28,71.01,82.0,0.0,5.99
424,griffith,au,-34.28,146.03,70.38,33.0,0.0,5.08
515,rancharia,br,-22.23,-50.89,72.37,76.0,0.0,8.34
527,leeton,au,-34.57,146.40,71.04,31.0,0.0,3.74
549,manaure,co,11.78,-72.44,76.95,76.0,0.0,2.48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["Cities","Country", "Latitude", "Longitude"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,Cities,Country,Latitude,Longitude,Hotel Name
135,bac lieu,vn,9.29,105.72,
165,cidreira,br,-30.18,-50.21,
185,taltal,cl,-25.40,-70.48,
254,mazabuka,zm,-15.86,27.75,
287,ballina,au,-28.87,153.57,
329,henties bay,na,-22.12,14.28,
424,griffith,au,-34.28,146.03,
515,rancharia,br,-22.23,-50.89,
527,leeton,au,-34.57,146.40,
549,manaure,co,11.78,-72.44,


In [20]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : api_key}

In [41]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["Cities"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
   # time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 60: port hedland.
Closest hotel in port hedland is The Esplanade Hotel.
------------
Retrieving Results for Index 84: busselton.
Closest hotel in busselton is Abbey Beach Resort.
------------
Retrieving Results for Index 151: carnarvon.
Closest hotel in carnarvon is Carnarvon Sea Change Apartments.
------------
Retrieving Results for Index 261: sibolga.
Closest hotel in sibolga is Prima Indah Hotel.
------------
Retrieving Results for Index 476: yulara.
Closest hotel in yulara is The Lost Camel Hotel - Ayers Rock Resort.
------------
-------End of Search-------


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Cities</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))